In [5]:
import scipy.io

In [7]:
cd C:/Users/haines/Dropbox/peach/dataproc/data/level1/b1

C:\Users\haines\Dropbox\peach\dataproc\data\level1\b1


In [11]:
data = scipy.io.loadmat('b1_ctd1_deployall.mat')
data.keys()

['salin',
 'qc',
 'press',
 '__header__',
 'gsw_SA',
 '__globals__',
 'platform',
 'depth',
 'cond',
 'dens',
 'mtime',
 'wtemp',
 '__version__',
 'config']

structures are a mess

In [25]:
data['config']

array([[ ([u'4085'], [[360]], [u'seconds'], [[array([u'mtime'], 
      dtype='<U5'), array([u'days since 0000-01-01T00:00'], 
      dtype='<U27')], [array([u'wtemp'], 
      dtype='<U5'), array([u'deg C'], 
      dtype='<U5')], [array([u'press'], 
      dtype='<U5'), array([u'decibars'], 
      dtype='<U8')], [array([u'cond'], 
      dtype='<U4'), array([u'S m-1'], 
      dtype='<U5')], [array([u'salin'], 
      dtype='<U5'), array([u'psu'], 
      dtype='<U3')], [array([u'depth'], 
      dtype='<U5'), array([u'm'], 
      dtype='<U1')], [array([u'dens'], 
      dtype='<U4'), array([u'kg m-3'], 
      dtype='<U6')]])]], 
      dtype=[('serial_number', 'O'), ('sample_interval', 'O'), ('sample_interval_units', 'O'), ('units', 'O')])

Trying using pytables or h5py

In [27]:
import tables

In [28]:
file = tables.openFile('b1_ctd1_deployall.mat')

HDF5ExtError: HDF5 error back trace

  File "..\..\hdf5-1.8.11\src\H5F.c", line 1582, in H5Fopen
    unable to open file
  File "..\..\hdf5-1.8.11\src\H5F.c", line 1373, in H5F_open
    unable to read superblock
  File "..\..\hdf5-1.8.11\src\H5Fsuper.c", line 334, in H5F_super_read
    unable to find file signature
  File "..\..\hdf5-1.8.11\src\H5Fsuper.c", line 155, in H5F_locate_signature
    unable to find a valid file signature

End of HDF5 error back trace

Unable to open/create file 'b1_ctd1_deployall.mat'